In [0]:
"""
A tours and travel company has 2 tables that relate to customers: FAMILIES and COUNTRIES . Each tour offers
a discount if a minimum number of people book at the same time.
Write a SQL query to print the maximum number of discounted tours any 1 family in the families table 
can choose from.

+----------------------------------+----------------+-------------+
| ID                               | NAME           | FAMILY_SIZE |
+----------------------------------+----------------+-------------+
| c00dac11bde74750b4d207b9c182a85f | Alex Thomas    |           9 |
| eb6f2d3426694667ae3e79d6274114a4 | Chris Gray     |           2 |
| 3f7b5b8e835d4e1c8b3e12e964a741f3 | Emily Johnson  |           4 |
| 9a345b079d9f4d3cafb2d4c11d20f8ce | Michael Brown  |           6 |
| e0a5f57516024de2a231d09de2cbe9d1 | Jessica Wilson |           3 |
+----------------------------------+----------------+-------------+
+----------------------------------+--------------+----------+----------+
| ID                               | NAME         | MIN_SIZE | MAX_SIZE |
+----------------------------------+--------------+----------+----------+
| 023fd23615bd4ff4b2ae0a13ed7efec9 | Bolivia      |        2 |        4 |
| be247f73de0f4b2d810367cb26941fb9 | Cook Islands |        4 |        8 |
| 3e85ab80a6f84ef3b9068b21dbcc54b3 | Brazil       |        4 |        7 |
| e571e164152c4f7c8413e2734f67b146 | Australia    |        5 |        9 |
| f35a7bb7d44342f7a8a42a53115294a8 | Canada       |        3 |        5 |
| a1b5a4b5fc5f46f891d9040566a78f27 | Japan        |       10 |       12 |
+----------------------------------+--------------+----------+----------+


Expected Output:
+----------------------+
| max_discounted_tours |
+----------------------+
|                    4 |
+----------------------+


/*
Output just for vizualization purpose, how data looks like for below query 

select * from FAMILIES f inner join COUNTRIES c
on f.FAMILY_SIZE between c.min_size and c.max_size ;
+----------------------------------+----------------+-------------+----------------------------------+--------------+----------+----------+
| ID                               | NAME           | FAMILY_SIZE | ID                               | NAME         | MIN_SIZE | MAX_SIZE |
+----------------------------------+----------------+-------------+----------------------------------+--------------+----------+----------+
| e0a5f57516024de2a231d09de2cbe9d1 | Jessica Wilson |           3 | 023fd23615bd4ff4b2ae0a13ed7efec9 | Bolivia      |        2 |        4 |
| 3f7b5b8e835d4e1c8b3e12e964a741f3 | Emily Johnson  |           4 | 023fd23615bd4ff4b2ae0a13ed7efec9 | Bolivia      |        2 |        4 |
| eb6f2d3426694667ae3e79d6274114a4 | Chris Gray     |           2 | 023fd23615bd4ff4b2ae0a13ed7efec9 | Bolivia      |        2 |        4 |
| 9a345b079d9f4d3cafb2d4c11d20f8ce | Michael Brown  |           6 | be247f73de0f4b2d810367cb26941fb9 | Cook Islands |        4 |        8 |
| 3f7b5b8e835d4e1c8b3e12e964a741f3 | Emily Johnson  |           4 | be247f73de0f4b2d810367cb26941fb9 | Cook Islands |        4 |        8 |
| 9a345b079d9f4d3cafb2d4c11d20f8ce | Michael Brown  |           6 | 3e85ab80a6f84ef3b9068b21dbcc54b3 | Brazil       |        4 |        7 |
| 3f7b5b8e835d4e1c8b3e12e964a741f3 | Emily Johnson  |           4 | 3e85ab80a6f84ef3b9068b21dbcc54b3 | Brazil       |        4 |        7 |
| 9a345b079d9f4d3cafb2d4c11d20f8ce | Michael Brown  |           6 | e571e164152c4f7c8413e2734f67b146 | Australia    |        5 |        9 |
| c00dac11bde74750b4d207b9c182a85f | Alex Thomas    |           9 | e571e164152c4f7c8413e2734f67b146 | Australia    |        5 |        9 |
| e0a5f57516024de2a231d09de2cbe9d1 | Jessica Wilson |           3 | f35a7bb7d44342f7a8a42a53115294a8 | Canada       |        3 |        5 |
| 3f7b5b8e835d4e1c8b3e12e964a741f3 | Emily Johnson  |           4 | f35a7bb7d44342f7a8a42a53115294a8 | Canada       |        3 |        5 |
+----------------------------------+----------------+-------------+----------------------------------+--------------+----------+----------+
*/

"""

from pyspark.sql.functions import *
from pyspark.sql.window import *

families_df = spark.createDataFrame([
    ('c00dac11bde74750b4d207b9c182a85f', 'Alex Thomas', 9),
    ('eb6f2d3426694667ae3e79d6274114a4', 'Chris Gray', 2),
    ('3f7b5b8e835d4e1c8b3e12e964a741f3', 'Emily Johnson', 4),
    ('9a345b079d9f4d3cafb2d4c11d20f8ce', 'Michael Brown', 6),
    ('e0a5f57516024de2a231d09de2cbe9d1', 'Jessica Wilson', 3)
], ["ID", "NAME", "FAMILY_SIZE"])

countries_df = spark.createDataFrame([
    ('023fd23615bd4ff4b2ae0a13ed7efec9', 'Bolivia', 2 , 4),
    ('be247f73de0f4b2d810367cb26941fb9', 'Cook Islands', 4,8),
    ('3e85ab80a6f84ef3b9068b21dbcc54b3', 'Brazil', 4,7),
    ('e571e164152c4f7c8413e2734f67b146', 'Australia', 5,9),
    ('f35a7bb7d44342f7a8a42a53115294a8', 'Canada', 3,5),
    ('a1b5a4b5fc5f46f891d9040566a78f27', 'Japan', 10,12)
], ["ID", "NAME", "MIN_SIZE", "MAX_SIZE"])

# families_df = families_df.withColumn("rn", row_number().over(Window.orderBy(lit(None))))
families_df.show(truncate=False)

# countries_df = countries_df.withColumn("rn", row_number().over(Window.orderBy(lit(None))))
countries_df.show(truncate=False)

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+--------------------------------+--------------+-----------+---+
|ID                              |NAME          |FAMILY_SIZE|rn |
+--------------------------------+--------------+-----------+---+
|c00dac11bde74750b4d207b9c182a85f|Alex Thomas   |9          |1  |
|eb6f2d3426694667ae3e79d6274114a4|Chris Gray    |2          |2  |
|3f7b5b8e835d4e1c8b3e12e964a741f3|Emily Johnson |4          |3  |
|9a345b079d9f4d3cafb2d4c11d20f8ce|Michael Brown |6          |4  |
|e0a5f57516024de2a231d09de2cbe9d1|Jessica Wilson|3          |5  |
+--------------------------------+--------------+-----------+---+

+--------------------------------+------------+--------+--------+---+
|ID                              |NAME        |MIN_SIZE|MAX_SIZE|rn |
+--------------------------------+------------+--------+--------+---+
|023fd23615bd4ff4b2ae0a13ed7efec9|Bolivia     |2       |4       |1  |
|be247f73de0f4b2d810367cb26941fb9|Cook Islands|4       |8       |2  |
|3e85ab80a6f84ef3b9068b21dbcc54b3|Brazil      |4       

In [0]:
families_df.createOrReplaceTempView("families")
countries_df.createOrReplaceTempView("countries")

spark.sql("""
          select max(cnt) as max_discounted_tours from (
            select 
                f.name, count(*) as cnt 
            from families f inner join countries c
            on f.FAMILY_SIZE between c.MIN_SIZE and c.MAX_SIZE
            group by f.name
          ) T
          """).show()

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+--------------------+
|max_discounted_tours|
+--------------------+
|                   4|
+--------------------+

